In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/consumer/Edureka_Consumer_Complaints_train.csv
/kaggle/input/consumer/Edureka_Consumer_Complaints_test.csv


In [2]:
df=pd.read_csv("/kaggle/input/consumer/Edureka_Consumer_Complaints_train.csv")
df.head(2)

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
0,2015-10-14,Credit reporting,NaN,Incorrect information on credit report,Information is not mine,NaN,NaN,Equifax,GA,30134,NaN,Consent not provided,Web,2015-10-14,Closed with explanation,Yes,No,1605653
1,2015-04-26,Bank account or service,Other bank product/service,Deposits and withdrawals,NaN,RE : XXXX XXXX XXXX-PRIVILEGED AND CONFIDENTIA...,NaN,Wells Fargo & Company,GA,319XX,NaN,Consent provided,Web,2015-04-26,Closed with explanation,Yes,Yes,1347613


In [3]:
df.shape

(358810, 18)

# **PART 1 - DATA EXPLORATION**

# **a• Analyze the missing values in the columns**

In [4]:
print("The total count of missing values is",df.isna().sum().sum())

The total count of missing values is 1488297


In [5]:
missing_val_col=pd.DataFrame(df.isna().sum())
missing_val_col.rename(columns={0:"missing_val_count"},inplace=True)
print("Out of",df.shape[1],"columns, below are the columns having missing values")
missing_val_col[~(missing_val_col["missing_val_count"]==0)]

Out of 18 columns, below are the columns having missing values


,missing_val_count
Sub-product,103786
Sub-issue,219374
Consumer complaint narrative,302630
Company public response,290879
State,2903
ZIP code,2911
Tags,308584
Consumer consent provided?,257230


In [6]:
nan_rows_df=pd.DataFrame(df.apply(lambda x: sum(x.isna()),axis=1))
nan_rows_count=nan_rows_df[~(nan_rows_df[0]==0)].count().tolist()[0]
print("Out of",df.shape[0],"rows,",nan_rows_count,"have missing values")

Out of 358810 rows, 357640 have missing values


# **b• What is the number of unique values and most frequently occurring categories in the Categorical Columns?**

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 358810 entries, 0 to 358809
Data columns (total 18 columns):
 #   Column                        Non-Null Count   Dtype 
---  ------                        --------------   ----- 
 0   Date received                 358810 non-null  object
 1   Product                       358810 non-null  object
 2   Sub-product                   255024 non-null  object
 3   Issue                         358810 non-null  object
 4   Sub-issue                     139436 non-null  object
 5   Consumer complaint narrative  56180 non-null   object
 6   Company public response       67931 non-null   object
 7   Company                       358810 non-null  object
 8   State                         355907 non-null  object
 9   ZIP code                      355899 non-null  object
 10  Tags                          50226 non-null   object
 11  Consumer consent provided?    101580 non-null  object
 12  Submitted via                 358810 non-null  object
 13 

In [8]:
column=df.columns.tolist()
##Dropping Complaint ID as it's of type int
column.remove("Complaint ID")

for i in column:
    print("# ",i)
    print("\t a. No of unique value is :",df[i].nunique() )
    print("\t b. Most frequently occuring",i,"is :",df[i].value_counts().reset_index()["index"].tolist()[0])

#  Date received
	 a. No of unique value is : 1758
	 b. Most frequently occuring Date received is : 2015-08-27
#  Product
	 a. No of unique value is : 12
	 b. Most frequently occuring Product is : Mortgage
#  Sub-product
	 a. No of unique value is : 47
	 b. Most frequently occuring Sub-product is : Other mortgage
#  Issue
	 a. No of unique value is : 95
	 b. Most frequently occuring Issue is : Loan modification,collection,foreclosure
#  Sub-issue
	 a. No of unique value is : 67
	 b. Most frequently occuring Sub-issue is : Account status
#  Consumer complaint narrative
	 a. No of unique value is : 55535
	 b. Most frequently occuring Consumer complaint narrative is : This company continues to report on my credit report after I sent them a letter telling them that this account was not mine and I have no idea what it is or who it belongs to! 

I asked for proof of a signed contract, I asked for a license to collect in my state, I asked for copies of all information referenced for this debt

# **c• Can you identify the top issues raised by consumers in this dataset?**

In [9]:
print("Below are the top 10 issues raised by consumers :")
df["Issue"].value_counts().reset_index().head(10)["index"]

Below are the top 10 issues raised by consumers :


0    Loan modification,collection,foreclosure
1      Incorrect information on credit report
2    Loan servicing, payments, escrow account
3       Cont'd attempts collect debt not owed
4     Account opening, closing, or management
5             Disclosure verification of debt
6                       Communication tactics
7                    Deposits and withdrawals
8    Application, originator, mortgage broker
9                            Billing disputes
Name: index, dtype: object

# **d• Are there some products which receive a higher number of complaints?**

In [10]:
print("Below are the top 7 products which receive a higher number of complaints :")
df["Product"].value_counts().reset_index().head(7)["index"]

Below are the top 7 products which receive a higher number of complaints :


0                   Mortgage
1            Debt collection
2           Credit reporting
3                Credit card
4    Bank account or service
5              Consumer Loan
6               Student loan
Name: index, dtype: object

# **e• Do all the companies receive the same number of complaints?**

In [11]:
df["Company"].value_counts()

Bank of America                     34903
Wells Fargo & Company               26490
JPMorgan Chase & Co.                21677
Equifax                             21489
Experian                            20075
                                    ...  
Nationwide Equities Corporation         1
New Southern Loans, Inc                 1
Nadler & Associates, P.C.               1
Law Offices of Vincent J. Buzek         1
Atlantis National Services, Inc.        1
Name: Company, Length: 3064, dtype: int64

*** From the above output, we can infer that, not same no of complaints is received by all the companies**

# **f• How are the complaints submitted - through which medium?**

In [12]:
print("Below are the medium for complaint registration :")
df["Submitted via"].unique().tolist()

Below are the medium for complaint registration :


['Web', 'Phone', 'Referral', 'Postal mail', 'Fax', 'Email']

# **g• What is the geographical distribution of the complaints?**

In [13]:
print("The complaints came from",df["State"].nunique(),"states and below are the list of reigons.")
df["State"].unique()

The complaints came from 62 states and below are the list of reigons.


array(['GA', 'SC', 'OH', 'CA', 'FL', 'NJ', 'IL', 'PA', 'ID', 'UT', 'TX',
       'TN', 'NM', 'VA', 'MO', 'KY', 'MN', 'IN', 'MD', 'WI', 'NY', 'KS',
       'CO', 'AZ', 'OK', 'MI', 'CT', 'LA', 'OR', 'NC', 'MS', 'NH', 'AL',
       'MA', 'WA', 'DC', 'AR', nan, 'VI', 'DE', 'NV', 'IA', 'WV', 'NE',
       'VT', 'ME', 'ND', 'HI', 'RI', 'WY', 'SD', 'PR', 'FM', 'MT', 'AK',
       'AE', 'GU', 'AP', 'MH', 'AS', 'MP', 'PW', 'AA'], dtype=object)

# **h• Do the complaints rise in any specific month or day of the week?**

In [14]:
df["month"]=df["Date received"].apply(lambda x: str(x).split("-")[1])
df.groupby("month")["month"].count().sort_values(ascending=False)

month
03    35578
04    34027
06    33635
05    33574
07    31431
01    30843
02    30672
08    28454
10    26257
12    25477
09    25393
11    23469
Name: month, dtype: int64

# **From the above output,complaint is at its peak in the month of March**

In [15]:
import datetime
import calendar
df["weekday"]=df["Date received"].apply(lambda date:calendar.day_name[datetime.datetime.strptime(date,"%Y-%m-%d").weekday()])
df.groupby("weekday")["weekday"].count().sort_values(ascending=False)

weekday
Wednesday    69583
Tuesday      68750
Thursday     66618
Friday       59348
Monday       58817
Saturday     20752
Sunday       14942
Name: weekday, dtype: int64

*** From the generated result, we can infer, maximum complaints were registered on Wednesday.**

#  **i• How do companies respond to the complaints? What are the most common responses received?**

In [16]:
df["Company public response"].value_counts()

Company chooses not to provide a public response                                                                           31526
Company has responded to the consumer and the CFPB and chooses not to provide a public response                            19268
Company believes it acted appropriately as authorized by contract or law                                                   12055
Company believes complaint caused principally by actions of third party outside the control or direction of the company     1091
Company believes the complaint is the result of a misunderstanding                                                          1075
Company disputes the facts presented in the complaint                                                                       1054
Company believes complaint is the result of an isolated error                                                                769
Company can't verify or dispute the facts in the complaint                                       

*** Above are companies response on complaints and top among all is "Company chooses not to provide a public response".**

# **j• Does responding to complaints in a timely manner alter the number of consumers that disputed the company response and those that did not?**

In [19]:
d1=df[df["Timely response?"]=="Yes"]
len(d1[d1["Consumer disputed?"]=="Yes"])

74943

In [20]:
len(d1[d1["Consumer disputed?"]=="No"])

277732

*** From the attained result, we infer that timely response does settle the dispute**